In [ ]:
pip install pyspark

In [ ]:
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, expr, struct

# 🔥 Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Filter Conversations") \
    .config("spark.driver.memory", "5G") \
    .config("spark.executor.memory", "5G") \
    .getOrCreate()

# 📂 Đọc file JSONL vào Spark DataFramea
file_path = "/opt/workspace/data.jsonl"
df = spark.read.json(file_path)

# 🛠️ Lọc các hội thoại có chứa assistant trả lời "can not solve"
df_filtered = df.filter(
    expr("array_contains(transform(messages, x -> lower(x.content)), 'can not solve')")
)

# 🔄 Sắp xếp lại thứ tự các field trong "messages"
df_reordered = df_filtered.withColumn(
    "messages", expr("transform(messages, x -> named_struct('role', x.role, 'content', x.content))")
)

# ✅ Kiểm tra dữ liệu
df_reordered.show(truncate=False)

# 📝 Lưu vào thư mục tạm với 1 file duy nhất
temp_output_path = "/opt/workspace/temp_output"
df_reordered.coalesce(1).write.mode("overwrite").json(temp_output_path)

# 📌 Lấy file JSON được tạo ra trong thư mục tạm
json_files = [f for f in os.listdir(temp_output_path) if f.endswith(".json")]

if json_files:
    json_file_path = os.path.join(temp_output_path, json_files[0])  # Lấy file JSON đầu tiên
    final_output_path = "/opt/workspace/cannot_solve_messages.jsonl"

    # 📌 Đổi tên file JSON thành JSONL
    shutil.move(json_file_path, final_output_path)

    # 🗑️ Xóa thư mục tạm để dọn dẹp
    shutil.rmtree(temp_output_path)

    print(f"✅ File đã được lưu tại: {final_output_path}")
else:
    print("❌ Không tìm thấy file JSON nào trong thư mục tạm!")
